![iceberg-logo](https://www.apache.org/logos/res/iceberg/iceberg.png)

# Getting Started with Iceberg and Spark

In [ ]:
spark

## Load One Month of NYC Taxi/Limousine Trip Data

For this notebook, we will use the New York City Taxi and Limousine Commision Trip Record Data that's available on the AWS Open Data Registry. This contains data of trips taken by taxis and for-hire vehicles in New York City. We'll save this into an iceberg table called `taxis`.

To be able to rerun the notebook several times, let's drop the table if it exists to start fresh.

In [ ]:
%%sql

DROP TABLE IF EXISTS taxis

In [ ]:
df = spark.read.option("header", True).option("inferSchema", "true").csv("/home/iceberg/data/yello_tripdata_2020-04.csv")
df.write.saveAsTable("taxis")

In [ ]:
%%sql

DESCRIBE EXTENDED taxis

In [ ]:
%%sql

SELECT COUNT(*) as cnt
FROM taxis

## Schema Evolution

Adding, dropping, renaming, or altering columns is easy and safe in Iceberg. In this example, we'll rename `fare_amount` to `fare` and `trip_distance` to `distance`. We'll also add a float column `fare_per_distance_unit` immediately after `distance`.

In [ ]:
%%sql

ALTER TABLE taxis
RENAME COLUMN fare_amount TO fare

In [ ]:
%%sql

ALTER TABLE taxis
RENAME COLUMN trip_distance TO distance

In [ ]:
%%sql

ALTER TABLE taxis
ADD COLUMN fare_per_distance_unit float AFTER distance

Let's update the new `fare_per_distance_unit` to equal `fare` divided by `distance`.

In [ ]:
%%sql

UPDATE taxis
SET fare_per_distance_unit = fare/distance

In [ ]:
%%sql

SELECT
VendorID
,tpep_pickup_datetime
,tpep_dropoff_datetime
,fare
,distance
,fare_per_distance_unit
FROM taxis

## Row Level Deletes
With Iceberg tables, `DELETE` queries can be used to perform row-level deletes. This is as simple as providing the table name and a `WHERE` predicate. If the filter matches an entire partition of the table, Iceberg will intelligently perform a metadata-only operation where it simply deletes the metadata for that partition.

Let's perform a row-level delete for all rows that have a `fare_per_distance_unit` greater than 4 and a `distance` greater than 2. This should leave us with relatively short trips that have a relatively high fare per distance traveled.

In [ ]:
%%sql

DELETE FROM taxis
WHERE fare_per_distance_unit > 4.0 OR distance > 2.0

There are some fares that have a `null` for `fare_per_distance_unit` due to the distance being `0`. Let's remove those as well.

In [ ]:
%%sql

DELETE FROM taxis
WHERE fare_per_distance_unit is null

In [ ]:
%%sql

SELECT
VendorID
,tpep_pickup_datetime
,tpep_dropoff_datetime
,fare
,distance
,fare_per_distance_unit
FROM taxis

In [ ]:
%%sql

SELECT COUNT(*) as cnt
FROM taxis

## Metadata Tables

Iceberg tables contain very rich metadata that can be easily queried. For example, you can retrieve the manifest list for any snapshot, simply by querying the table's `snapshots` table.

In [ ]:
%%sql

SELECT snapshot_id, manifest_list
FROM taxis.snapshots

The `files` table contains loads of information on data files, including column level statistics such as null counts, lower bounds, and upper bounds.

In [ ]:
%%sql

SELECT file_path, file_format, record_count, null_value_counts, lower_bounds, upper_bounds
FROM taxis.files

## Time Travel

The history table lists all snapshots and which parent snapshot they derive from. The `is_current_ancestor` flag let's you know if a snapshot is part of the linear history of the current snapshot of the table.

In [ ]:
%%sql

SELECT *
FROM taxis.history

You can time-travel by altering the `current-snapshot-id` property of the table to reference any snapshot in the table's history. Let's revert the table to it's original state by traveling to the very first snapshot ID.

In [ ]:
%%sql --var df

SELECT *
FROM taxis.history

In [ ]:
original_snapshot = df.head().snapshot_id
print(original_snapshot)
spark.sql(f"ALTER TABLE taxis SET TBLPROPERTIES ('current-snapshot-id'={original_snapshot})")

In [ ]:
%%sql

SELECT
VendorID
,tpep_pickup_datetime
,tpep_dropoff_datetime
,fare
,distance
,fare_per_distance_unit
FROM taxis

Another look at the history table shows that the original state of the table has been added as a new entry
with the original snapshot ID.

In [ ]:
%%sql

SELECT *
FROM taxis.history

In [ ]:
%%sql

SELECT COUNT(*) as cnt
FROM taxis